This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# xradar furuno backend

In this example, we read scn/scnx (furuno) data files using the xradar `furuno` xarray backend.

In [ ]:
import glob
import gzip
import io
import wradlib as wrl
import warnings

warnings.filterwarnings("ignore")
import matplotlib.pyplot as pl
import numpy as np
import xradar as xd
import datatree as xt
import xarray as xr

try:
    get_ipython().run_line_magic("matplotlib inline")
except:
    pl.ion()

## Load furuno scn Data

Data provided by University of Graz, Austria.

In [ ]:
fpath = "furuno/0080_20210730_160000_01_02.scn.gz"
f = wrl.util.get_wradlib_data_file(fpath)
vol = xd.io.open_furuno_datatree(f, reindex_angle=False)

### Inspect RadarVolume

In [ ]:
display(vol)

### Inspect root group

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
vol.root

### Inspect sweep group(s)

The sweep-groups can be accessed via their respective keys. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
display(vol["sweep_0"])

### Goereferencing

In [ ]:
swp = vol["sweep_0"].ds.copy()
swp = swp.assign_coords(sweep_mode=swp.sweep_mode)
swp = swp.pipe(wrl.georef.georeference_dataset)

### Plotting

Currently the data dynamic range is left as read from the file. That way the difference between shortpulse and longpulse can be clearly seen.

In [ ]:
swp.DBZH.plot.pcolormesh(x="x", y="y")
pl.gca().set_aspect("equal")

In [ ]:
fig = pl.figure(figsize=(10, 10))
swp.DBZH.wrl.vis.plot(proj="cg", fig=fig)

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
pm = swp.DBZH.wrl.vis.plot(proj=map_proj)
ax = pl.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp.DBZH.wrl.vis.plot(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
import cartopy.feature as cfeature


def plot_rivers(ax):
    rivers = cfeature.NaturalEarthFeature(
        category="physical",
        name="rivers_lake_centerlines",
        scale="10m",
        facecolor="none",
    )
    ax.add_feature(rivers, edgecolor="blue", lw=2, zorder=4)


map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)

DBZH = swp.DBZH
pm = DBZH.where(DBZH > 0).wrl.vis.plot(ax=ax)
plot_rivers(ax)
ax.gridlines(draw_labels=True)

In [ ]:
import matplotlib.path as mpath

theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values,
    central_longitude=swp.longitude.values,
)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
ax.set_boundary(circle, transform=ax.transAxes)

pm = swp.DBZH.wrl.vis.plot(proj=map_proj, ax=ax)
ax = pl.gca()
ax.gridlines(crs=map_proj)

In [ ]:
fig = pl.figure(figsize=(10, 8))
proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
ax = fig.add_subplot(111, projection=proj)
pm = swp.DBZH.wrl.vis.plot(ax=ax)
ax.gridlines()

In [ ]:
swp.DBZH.wrl.vis.plot()

### Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray has dimensions and coordinates of it's parent dataset.

In [ ]:
display(swp.DBZH)

### Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('time')` method, which sorts the radials by time.

In [ ]:
swp.DBZH.sortby("time").plot(x="range", y="time", add_labels=False)

In [ ]:
fig = pl.figure(figsize=(5, 5))
pm = swp.DBZH.wrl.vis.plot(proj={"latmin": 3e3}, fig=fig)

### Mask some values

In [ ]:
dbzh = swp["DBZH"].where(swp["DBZH"] >= 0)
dbzh.plot(x="x", y="y")

### Export to ODIM and CfRadial2


In [ ]:
xd.io.to_odim(vol, "furuno_scn_as_odim.h5")
xd.io.to_cfradial2(vol, "furuno_scn_as_cfradial2.nc")

### Import again

In [ ]:
vola = xd.io.open_odim_datatree("furuno_scn_as_odim.h5", reindex_angle=False)
display(vola)

In [ ]:
volb = xt.open_datatree("furuno_scn_as_cfradial2.nc")
display(volb)

### Use `xr.open_dataset` to retrieve explicit group

## Load furuno scnx Data

Data provided by GFZ German Research Centre for Geosciences.

In [ ]:
fpath = "furuno/2006_20220324_000000_000.scnx.gz"
f = wrl.util.get_wradlib_data_file(fpath)
vol = xd.io.open_furuno_datatree(f, reindex_angle=False)

### Inspect RadarVolume

In [ ]:
display(vol)

### Inspect root group

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
vol.root

### Inspect sweep group(s)

The sweep-groups can be accessed via their respective keys. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
display(vol["sweep_0"])

### Goereferencing

In [ ]:
swp = vol["sweep_0"].ds.copy()
swp = swp.assign_coords(sweep_mode=swp.sweep_mode)
swp = swp.pipe(wrl.georef.georeference_dataset)

### Plotting

Currently the data dynamic range is left as read from the file. That way the difference between shortpulse and longpulse can be clearly seen.

In [ ]:
swp.DBZH.plot.pcolormesh(x="x", y="y")
pl.gca().set_aspect("equal")

In [ ]:
fig = pl.figure(figsize=(10, 10))
swp.DBZH.wrl.vis.plot(proj="cg", fig=fig)

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
pm = swp.DBZH.wrl.vis.plot(proj=map_proj)
ax = pl.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp.DBZH.wrl.vis.plot(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
import cartopy.feature as cfeature


def plot_rivers(ax):
    rivers = cfeature.NaturalEarthFeature(
        category="physical",
        name="rivers_lake_centerlines",
        scale="10m",
        facecolor="none",
    )
    ax.add_feature(rivers, edgecolor="blue", lw=2, zorder=4)


map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)

DBZH = swp.DBZH
pm = DBZH.where(DBZH > 0).wrl.vis.plot(ax=ax)
plot_rivers(ax)
ax.gridlines(draw_labels=True)

In [ ]:
import matplotlib.path as mpath

theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values,
    central_longitude=swp.longitude.values,
)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
ax.set_boundary(circle, transform=ax.transAxes)

pm = swp.DBZH.wrl.vis.plot(proj=map_proj, ax=ax)
ax = pl.gca()
ax.gridlines(crs=map_proj)

In [ ]:
fig = pl.figure(figsize=(10, 8))
proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
ax = fig.add_subplot(111, projection=proj)
pm = swp.DBZH.wrl.vis.plot(ax=ax)
ax.gridlines()

In [ ]:
swp.DBZH.wrl.vis.plot()

### Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray has dimensions and coordinates of it's parent dataset.

In [ ]:
display(swp.DBZH)

### Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('time')` method, which sorts the radials by time.

In [ ]:
swp.DBZH.sortby("time").plot(x="range", y="time", add_labels=False)

In [ ]:
fig = pl.figure(figsize=(5, 5))
pm = swp.DBZH.wrl.vis.plot(proj={"latmin": 3e3}, fig=fig)

### Mask some values

In [ ]:
dbzh = swp["DBZH"].where(swp["DBZH"] >= 0)
dbzh.plot(x="x", y="y")

### Export to ODIM and CfRadial2


In [ ]:
xd.io.to_odim(vol, "furuno_scnx_as_odim.h5")
xd.io.to_cfradial2(vol, "furuno_scnx_as_cfradial2.nc")

### Import again

In [ ]:
vola = xd.io.open_odim_datatree("furuno_scnx_as_odim.h5", reindex_angle=False)
display(vola)

In [ ]:
volb = xt.open_datatree("furuno_scnx_as_cfradial2.nc")
display(volb)

## More Furuno loading mechanisms

### Use `xr.open_dataset` to retrieve explicit group

In [ ]:
swp_b = xr.open_dataset(f, engine="furuno", backend_kwargs=dict(reindex_angle=False))
display(swp_b)